In [1]:
import pandas as pd
import numpy as np
pd.set_option("mode.copy_on_write", True)

In [4]:
import getpass
pg_password = getpass.getpass()

 ········


In [5]:
import psycopg2
pg_dsn = f"dbname=replicator user=replicator host=127.0.0.1 port=6541 password={pg_password}"
tmp_sql = "select now()"
with psycopg2.connect(pg_dsn) as conn:
    with conn.cursor() as cursor:
        cursor.execute(tmp_sql)
        print(cursor.fetchone())

(datetime.datetime(2024, 3, 10, 1, 49, 29, 881239, tzinfo=datetime.timezone.utc),)


In [6]:
import tempfile
def ijv_df_read_sql_tmpfile(query):
    with tempfile.TemporaryFile() as tmpfile:
        copy_sql = "COPY ({query}) TO STDOUT WITH CSV {head}".format(
           query=query, head="HEADER"
        )
        with psycopg2.connect(pg_dsn) as conn:
            with conn.cursor() as cursor:
                cursor.copy_expert(copy_sql, tmpfile)
                tmpfile.seek(0)
                df = pd.read_csv(tmpfile, index_col=['i','j'], dtype={'i': 'Int32', 'j': 'Int32'})
                return df

In [7]:
%%time
likes_sql = """
    SELECT fid as i, target_cast_fid as j, count(1) as likes_v 
    FROM reactions 
    WHERE type=1
    AND fid < 10000
    GROUP BY i, j
    -- limit 100
  """
likes_df = ijv_df_read_sql_tmpfile(likes_sql)

CPU times: user 585 ms, sys: 69.5 ms, total: 654 ms
Wall time: 4.75 s


In [8]:
likes_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 271944 entries, (2526, 6946) to (2433, 56)
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   likes_v  271944 non-null  int64
dtypes: int64(1)
memory usage: 3.7 MB


In [9]:
likes_df.sample(5)

,,likes_v
i,j,
2056,191824,1
473,1030,2
4753,4612,1
5543,191775,4
4565,4407,1


In [10]:
likes_df.index.dtype

dtype('O')

In [11]:
%%time
replies_sql = """
    SELECT fid as i, parent_fid as j, count(1) as replies_v 
    FROM casts
    WHERE parent_hash IS NOT NULL
    AND fid < 10000
    GROUP by i, j
  """
replies_df = ijv_df_read_sql_tmpfile(replies_sql)

CPU times: user 497 ms, sys: 61.6 ms, total: 559 ms
Wall time: 4.8 s


In [12]:
replies_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 231879 entries, (4651, 431) to (8149, 7143)
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   replies_v  231879 non-null  int64
dtypes: int64(1)
memory usage: 3.2 MB


In [13]:
%%time
mentions_sql = """
		WITH mention AS (
			SELECT fid as author_fid, mention.value as mention_fid 
			FROM casts, json_array_elements_text(casts.mentions) as mention
		)
		SELECT 
			author_fid as i, mention_fid as j, count(1) as mentions_v
		FROM mention
        WHERE author_fid < 10000
		GROUP BY i, j
  """
mentions_df = ijv_df_read_sql_tmpfile(mentions_sql)

CPU times: user 143 ms, sys: 22.2 ms, total: 165 ms
Wall time: 3.21 s


In [14]:
mentions_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 48679 entries, (2, 1) to (9994, 3)
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   mentions_v  48679 non-null  int64
dtypes: int64(1)
memory usage: 918.8 KB


In [15]:
%%time
recasts_sql = """
    SELECT fid as i, target_cast_fid as j, count(1) as reacts_v 
    FROM reactions 
    WHERE type=2
    AND fid < 10000
    GROUP BY i, j
  """
recasts_df = ijv_df_read_sql_tmpfile(recasts_sql)

CPU times: user 147 ms, sys: 25.9 ms, total: 173 ms
Wall time: 2.76 s


In [16]:
recasts_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 43143 entries, (2, 2) to (9991, 189625)
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   reacts_v  43143 non-null  int64
dtypes: int64(1)
memory usage: 686.5 KB


In [17]:
%%time
follows_sql = """
    SELECT 
        follower_fid as i, 
        following_fid as j,
        1 as follows_v
    FROM mv_follow_links 
    WHERE follower_fid < 10000
    ORDER BY i, j, follows_v desc
    """
follows_df = ijv_df_read_sql_tmpfile(follows_sql)

CPU times: user 2.32 s, sys: 242 ms, total: 2.57 s
Wall time: 21 s


In [18]:
follows_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1100115 entries, (1, 2) to (9999, 9892)
Data columns (total 1 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   follows_v  1100115 non-null  int64
dtypes: int64(1)
memory usage: 15.9 MB


In [19]:
%%time
df = likes_df.merge(replies_df, how='outer', left_index=True, right_index=True, indicator=False)

CPU times: user 330 ms, sys: 42 ms, total: 372 ms
Wall time: 394 ms


In [20]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 403229 entries, (1, 3) to (9998, 970)
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   likes_v    271944 non-null  float64
 1   replies_v  231879 non-null  float64
dtypes: float64(2)
memory usage: 8.3 MB


In [21]:
%%time
df = df.merge(mentions_df, how='outer', left_index=True, right_index=True, indicator=False)

CPU times: user 265 ms, sys: 41.7 ms, total: 307 ms
Wall time: 324 ms


In [22]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 421416 entries, (1, 3) to (9998, 970)
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   likes_v     271944 non-null  float64
 1   replies_v   231879 non-null  float64
 2   mentions_v  48679 non-null   float64
dtypes: float64(3)
memory usage: 11.9 MB


In [23]:
%%time
df = df.merge(recasts_df, how='outer', left_index=True, right_index=True, indicator=False)

CPU times: user 271 ms, sys: 54 ms, total: 325 ms
Wall time: 348 ms


In [24]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 424852 entries, (1, 3) to (9998, 970)
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   likes_v     271944 non-null  float64
 1   replies_v   231879 non-null  float64
 2   mentions_v  48679 non-null   float64
 3   reacts_v    43143 non-null   float64
dtypes: float64(4)
memory usage: 15.2 MB


In [25]:
%%time
df = df.merge(follows_df, how='outer', left_index=True, right_index=True, indicator=False)

CPU times: user 897 ms, sys: 182 ms, total: 1.08 s
Wall time: 1.24 s


In [26]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1303353 entries, (1, 2) to (9999, 9892)
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   likes_v     271944 non-null   float64
 1   replies_v   231879 non-null   float64
 2   mentions_v  48679 non-null    float64
 3   reacts_v    43143 non-null    float64
 4   follows_v   1100115 non-null  float64
dtypes: float64(5)
memory usage: 58.4 MB


In [27]:
df.sample(10)

,,likes_v,replies_v,mentions_v,reacts_v,follows_v
i,j,,,,,
3866,4091,NaN,NaN,NaN,NaN,1.0
716,4341,NaN,1.0,NaN,NaN,NaN
424,27,NaN,NaN,NaN,NaN,1.0
283,578,NaN,NaN,NaN,NaN,1.0
3514,463,NaN,NaN,NaN,NaN,1.0
8062,194,NaN,NaN,NaN,NaN,1.0
9523,1287,NaN,NaN,NaN,NaN,1.0
8106,60,NaN,NaN,NaN,NaN,1.0
7675,1655,NaN,NaN,NaN,NaN,1.0
